<a href="https://colab.research.google.com/github/wenqilyu/Capstone/blob/main/clean_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/MyDrive/Capstone

import re
import pandas as pd
import string

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/MyDrive/Capstone


In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 13.6 gigabytes of available RAM

Not using a high-RAM runtime


## Review Data

In [ ]:
all_tweets = pd.read_csv('all_id_combined.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
all_tweets.head()

,coordinates,created_at,hashtags,media,urls,favorite_count,id,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_user_id,lang,place,possibly_sensitive,quote_id,retweet_count,retweet_id,retweet_screen_name,source,text,tweet_url,user_created_at,user_id,user_default_profile_image,user_description,user_favourites_count,user_followers_count,user_friends_count,user_listed_count,user_location,user_name,user_screen_name,user_statuses_count,user_time_zone,user_urls,user_verified
0,NaN,Tue Jul 14 14:28:11 +0000 2020,NaN,NaN,NaN,0,1283045634879234051,NaN,NaN,NaN,en,NaN,NaN,1.282771e+18,1,1.283038e+18,HellAnnBack,"<a href=""http://twitter.com/download/android"" ...","Mask nazis? What a guy, this mayor. But this i...",https://twitter.com/jellison_pat/status/128304...,Sun Jul 28 17:11:46 +0000 2019,1155526269255241728,False,"Sense of humor, think Far Side. Thinks outside...",83756,784,823,0,NaN,Pat Jellison,jellison_pat,27989,NaN,NaN,False
1,NaN,Tue Jul 14 14:28:11 +0000 2020,NaN,NaN,NaN,0,1283045635235745794,NaN,NaN,NaN,en,NaN,NaN,1.282983e+18,111,1.283043e+18,RLHeinrichs,"<a href=""http://twitter.com/download/iphone"" r...","Uh. You guys. “Orlando Health, the only to ans...",https://twitter.com/ladyoutlander72/status/128...,Sun Feb 25 21:22:14 +0000 2018,967872389274112006,False,"Constitutional Conservative, wife, mother, mem...",5555,5986,6104,1,"Texas, USA",Kricket Dalais,ladyoutlander72,23356,NaN,NaN,False
2,NaN,Mon Jul 13 09:47:32 +0000 2020,COVID19,NaN,https://twitter.com/catalayahere/status/128241...,1,1282612619497676800,NaN,NaN,NaN,en,NaN,False,1.282418e+18,0,NaN,NaN,"<a href=""http://twitter.com/download/android"" ...",Eye opening thread about #COVID19 and 1% mort...,https://twitter.com/DrMphoMalatji/status/12826...,Mon Sep 20 22:33:46 +0000 2010,193077115,False,•Philosopher\n•Indigenous Knowledge Librarian\...,3590,4414,3609,1,Johannesburg,Dr Mpho Malatji,DrMphoMalatji,7099,NaN,NaN,False
3,NaN,Mon Jul 13 09:38:04 +0000 2020,NaN,NaN,NaN,0,1282610233714585600,NaN,NaN,NaN,en,NaN,NaN,NaN,99,1.282582e+18,ABC,"<a href=""http://twitter.com/download/iphone"" r...","More than six months into the global pandemic,...",https://twitter.com/ksev4/status/1282610233714...,Fri Apr 01 22:05:36 +0000 2011,275735030,False,My heart wants to sigh like a chime that flies...,732909,1447,718,30,"Wells, Maine",Katherine H Sevigney,ksev4,143442,NaN,http://KHSJourneyTo30.weebly.com,False
4,NaN,Tue Jul 14 14:55:31 +0000 2020,NaN,NaN,https://twitter.com/Fox35Amy/status/1282982855...,0,1283052510689800192,NaN,NaN,NaN,en,NaN,False,1.282983e+18,425,1.283045e+18,atensnut,"<a href=""https://mobile.twitter.com"" rel=""nofo...",Paging @GovRonDeSantis https://t.co/usrBnKOLAH,https://twitter.com/katsnbears/status/12830525...,Fri Aug 16 00:32:04 +0000 2013,1674389960,False,"🍊🇺🇸❤️#MAGA, #Patriot #LovemyCountry #TrumpWon!...",50728,5117,5113,1,Beautiful state of Tennessee,Anne,katsnbears,42021,NaN,NaN,False


In [ ]:
all_tweets.columns.values

array(['coordinates', 'created_at', 'hashtags', 'media', 'urls',
       'favorite_count', 'id', 'in_reply_to_screen_name',
       'in_reply_to_status_id', 'in_reply_to_user_id', 'lang', 'place',
       'possibly_sensitive', 'quote_id', 'retweet_count', 'retweet_id',
       'retweet_screen_name', 'source', 'text', 'tweet_url',
       'user_created_at', 'user_id', 'user_default_profile_image',
       'user_description', 'user_favourites_count',
       'user_followers_count', 'user_friends_count', 'user_listed_count',
       'user_location', 'user_name', 'user_screen_name',
       'user_statuses_count', 'user_time_zone', 'user_urls',
       'user_verified'], dtype=object)

## Process Data

In [ ]:
puncts = list(string.punctuation)

In [ ]:
def clean_data(text):
  # delete punctuation
  for punc in puncts:
    if punc in text:
      text = text.replace(punc, f'')
  #make the text lower case
  text=text.lower()
  # delete the url
  text=re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '', text)
  # delete the email
  text=re.sub(r'^([\w]+\.*)([\w]+)\@[\w]+\.\w{3}(\.\w{2}|)', '', text)
  # delete @someone
  text=re.sub(r'@\S+', '', text)
  # delete #tags
  text=re.sub(r'#\S+', '', text)
  # delete \n
  text=re.sub(r'^\s*$(?:\n|\r\n)', '', text)
  return text

In [ ]:
all_tweets['text']=all_tweets['text'].apply(clean_data)

In [ ]:
all_tweets['text']

## Delete Stop words Data

In [ ]:
! pip install spacy
! pip install spacy_cld
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.language import Language
from spacy_cld import LanguageDetector
nlp = spacy.load('en')
language_detector = LanguageDetector()
nlp.add_pipe(language_detector)


     |████████████████████████████████| 41.4 MB 1.3 MB/s 
  Created wheel for spacy-cld: filename=spacy_cld-0.1.0-py3-none-any.whl size=4065 sha256=307b3d7effe32b0753dbdf04cbe796dcd157eb9c480a126a3089de24b8a781de
  Stored in directory: /root/.cache/pip/wheels/82/6b/8f/260626615a2629391aca1fce5d3c676dde154f8aabb18d25bf
  Created wheel for pycld2: filename=pycld2-0.41-cp37-cp37m-linux_x86_64.whl size=9834387 sha256=9f4d06c87136100e550fa822dc1422bffbba92a2f205947a9183a804afd3292f
  Stored in directory: /root/.cache/pip/wheels/ed/e4/58/ed2e9f43c07d617cc81fe7aff0fc6e42b16c9cf6afe960b614
Successfully built spacy-cld pycld2


In [ ]:
all_tweets['language']='unknown'

In [ ]:
for i in range(len(all_tweets)):
  try:
    doc = nlp(all_tweets['text'][i])
    all_tweets['language'][i]=doc._.languages
  except:
    all_tweets['language'][i]='failed'

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
all_tweets['language']

In [ ]:
nlp = spacy.load('en_core_web_sm')

In [ ]:
stopwords=list(STOP_WORDS)

for word in stopwords:
    nlp.vocab[word].is_stop = True

In [ ]:
all_tweets['text'] = all_tweets['text'].apply(lambda text: 
                                          " ".join(token.lemma_ for token in nlp(text) 
                                                   if not token.is_stop))

# Detect English

In [ ]:
#Delete Null
all_tweets=all_tweets[all_tweets['text'].isnull()==False]

In [ ]:
! pip install langdetect
import langdetect

In [ ]:
#Detect English
all_tweets['Language'] = 'unknown'
for i in range(len(all_tweets)):
  try:
    if langdetect.detect(all_tweets['text'][i])=='en':
      all_tweets['Language'][i]='en'
  except:
    all_tweets['Language'][i]='delete'

In [ ]:
all_tweets.to_csv("output2.csv")

In [ ]:
all_tweets['text']

In [ ]:
pd.read_csv('output.csv').head()